Аветісова Карина
## Аналіз A/B-тестів

Ви - аналітик даних в ІТ-компанії і до вас надійшла задача проаналізувати дані A/B тесту в популярній [грі Cookie Cats](https://www.facebook.com/cookiecatsgame). Це - гра-головоломка в стилі «з’єднай три», де гравець повинен з’єднати плитки одного кольору, щоб очистити дошку та виграти рівень. На дошці також зображені співаючі котики :)

Під час проходження гри гравці стикаються з воротами, які змушують їх чекати деякий час, перш ніж вони зможуть прогресувати або зробити покупку в додатку.

У цьому блоці завдань ми проаналізуємо результати A/B тесту, коли перші ворота в Cookie Cats було переміщено з рівня 30 на рівень 40. Зокрема, ми хочемо зрозуміти, як це вплинуло на утримання (retention) гравців. Тобто хочемо зрозуміти, чи переміщення воріт на 10 рівнів пізніше якимось чином вплинуло на те, що користувачі перестають грати в гру раніше чи пізніше з точки зору кількості їх днів з моменту встановлення гри.

Будемо працювати з даними з файлу `cookie_cats.csv`. Колонки в даних наступні:

- `userid` - унікальний номер, який ідентифікує кожного гравця.
- `version` - чи потрапив гравець в контрольну групу (gate_30 - ворота на 30 рівні) чи тестову групу (gate_40 - ворота на 40 рівні).
- `sum_gamerounds` - кількість ігрових раундів, зіграних гравцем протягом першого тижня після встановлення
- `retention_1` - чи через 1 день після встановлення гравець повернувся і почав грати?
- `retention_7` - чи через 7 днів після встановлення гравець повернувся і почав грати?

Коли гравець встановлював гру, його випадковим чином призначали до групи gate_30 або gate_40.

1. Для початку, уявімо, що ми тільки плануємо проведення зазначеного А/B-тесту і хочемо зрозуміти, дані про скількох користувачів нам треба зібрати, аби досягнути відчутного ефекту. Відчутним ефектом ми вважатимемо збільшення утримання на 1% після внесення зміни. Обчисліть, скільки користувачів сумарно нам треба аби досягнути такого ефекту, якщо продакт менеджер нам повідомив, що базове утримання є 19%.

In [2]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.stats.api as sms
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from math import ceil
%matplotlib inline

In [14]:
baseline_rate = 0.19
effect_size = 0.01
alpha = 0.05
power = 0.8

es = sms.proportion_effectsize(0.19, 0.20)

analysis = sms.NormalIndPower()
sample_size_per_group = analysis.solve_power(effect_size=es, power=power, alpha=alpha, ratio=1)

print("Необхідний розмір вибірки на групу:", ceil(sample_size_per_group))
print("Сумарний розмір вибірки:", ceil(sample_size_per_group * 2))

Необхідний розмір вибірки на групу: 24638
Сумарний розмір вибірки: 49276


2. Зчитайте дані АВ тесту у змінну `df` та виведіть середнє значення показника показник `retention_7` (утримання на 7 день) по версіям гри. Сформулюйте гіпотезу: яка версія дає краще утримання через 7 днів після встановлення гри?

In [13]:
df = pd.read_csv("cookie_cats.csv")
means = df.groupby("version")["retention_7"].mean()
print("Середні значення retention_7 по версіях:")
print(means)

Середні значення retention_7 по версіях:
version
gate_30    0.190201
gate_40    0.182000
Name: retention_7, dtype: float64


gate_30 (контрольна група) → середнє утримання ≈ 0.1902 (19.0%);

gate_40 (тестова група) → середнє утримання ≈ 0.1820 (18.2%)

Нульова гіпотеза (H₀): середнє утримання через 7 днів не залежить від версії гри.
p(gate_30) = p(gate_40)

Альтернативна гіпотеза (H₀): середнє утримання через 7 днів не залежить від версії гри.
p(gate_30) > p(gate_40)

Виходячи з отриманих значень середнього отримання gate_30 дає кращі показники.

3. Перевірте з допомогою пасуючого варіанту z-тесту, чи дає якась з версій гри кращий показник `retention_7` на рівні значущості 0.05. Обчисліть також довірчі інтервали для варіантів до переміщення воріт і після. Виведіть результат у форматі:

    ```
    z statistic: ...
    p-value: ...
    Довірчий інтервал 95% для групи control: [..., ...]
    Довірчий інтервал 95% для групи treatment: [..., ...]
    ```

    де замість `...` - обчислені значення.
    
    В якості висновку дайте відповідь на два питання:  

      1. Чи є статистична значущою різниця між поведінкою користувачів у різних версіях гри?   
      2. Чи перетинаються довірчі інтервали утримання користувачів з різних версій гри? Про що це каже?  


In [17]:
from statsmodels.stats.proportion import proportions_ztest, proportion_confint


grouped = df.groupby("version")["retention_7"]
successes = grouped.sum().values 
nobs = grouped.count().values

z_stat, p_val = proportions_ztest(successes, nobs)
print("z statistic:", z_stat)
print("p-value:", p_val)


conf_int_control = proportion_confint(successes[0], nobs[0], alpha=0.05, method='normal')
conf_int_treatment = proportion_confint(successes[1], nobs[1], alpha=0.05, method='normal')

print(f"Довірчий інтервал 95% для групи control: {conf_int_control}")
print(f"Довірчий інтервал 95% для групи treatment: {conf_int_treatment}")

z statistic: 3.164358912748191
p-value: 0.001554249975614329
Довірчий інтервал 95% для групи control: (0.18656311652199903, 0.19383956804175934)
Довірчий інтервал 95% для групи treatment: (0.17845430073314686, 0.18554578720019968)


Оскільки p-value < z-stat, на рівні значущості 5% ми відхиляємо гіпотезу Н0.
Отже, різниця в утриманні між версіями статистично значуща.

Довірчі інтервали не перетинаються. Це каже про те, що навіть з урахуванням статистичної похибки утримання в контрольній групі вище, ніж у тестовій.

Як висновок, переміщення воріт з 30-го на 40-й рівень негативно вплинуло на утримання користувачів на 7-й день. Контрольна версія (gate_30) показує краще утримання.

4. Виконайте тест Хі-квадрат на рівні значущості 5% аби визначити, чи є залежність між версією гри та утриманням гравця на 7ий день після реєстрації.

    - Напишіть, як для цього тесту будуть сформульовані гіпотези.
    - Проведіть обчислення, виведіть p-значення і напишіть висновок за результатами тесту.


In [20]:
from scipy.stats import chi2_contingency

contingency_table = pd.crosstab(df["version"], df["retention_7"])

chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Chi-square statistic:", chi2)
print("p-value:", p)
print("Ступені свободи:", dof)
print("Очікувані частоти:")
print(expected)

Chi-square statistic: 9.959086799559167
p-value: 0.0016005742679058301
Ступені свободи: 1
Очікувані частоти:
[[36382.90257127  8317.09742873]
 [37025.09742873  8463.90257127]]


Формулювання гіпотез:

Н0: версія гри (gate_30 vs gate_40) не впливає на утримання користувачів через 7 днів. Розподіл подій (утримався / не утримався) однаковий у двох групах.

На: версія гри впливає на утримання користувачів (розподіли відрізняються).

Оскільки p-value = 0.0016 < 0.05, ми відхиляємо нульову гіпотезу.
Це означає, що різниця між версіями гри є статистично значущою: утримання користувачів дійсно відрізняється між gate_30 та gate_40.